In [3]:
import pandas as pd
import time
import os
import sys

SPIDER_DATASET_DIR = "../dataset/spider/"

In [54]:
def find_primary_keys_MYSQL_like(db_name, spider_primary):
    df = spider_primary[spider_primary['Database name'] == db_name]
    output = "["
    for index, row in df.iterrows():
        output += row['Table Name'] + '.' + row['Primary Key'] +','
    output = output[:-1]
    output += "]\n"
    return output
def creating_schema(DATASET_JSON):
    schema_df = pd.read_json(DATASET_JSON)
    schema_df = schema_df.drop(['column_names','table_names'], axis=1)
    schema = []
    f_keys = []
    p_keys = []
    for index, row in schema_df.iterrows():
        tables = row['table_names_original']
        col_names = row['column_names_original']
        col_types = row['column_types']
        foreign_keys = row['foreign_keys']
        primary_keys = row['primary_keys']
        for col, col_type in zip(col_names, col_types):
            index, col_name = col
            if index == -1:
                for table in tables:
                    schema.append([row['db_id'], table, '*', 'text'])
            else:
                schema.append([row['db_id'], tables[index], col_name, col_type])
        for primary_key in primary_keys:
            index, column = col_names[primary_key]
            p_keys.append([row['db_id'], tables[index], column])
        for foreign_key in foreign_keys:
            first, second = foreign_key
            first_index, first_column = col_names[first]
            second_index, second_column = col_names[second]
            f_keys.append([row['db_id'], tables[first_index], tables[second_index], first_column, second_column])
    spider_schema = pd.DataFrame(schema, columns=['Database name', ' Table Name', ' Field Name', ' Type'])
    spider_primary = pd.DataFrame(p_keys, columns=['Database name', 'Table Name', 'Primary Key'])
    spider_foreign = pd.DataFrame(
        f_keys,
        columns=[
            'Database name', 'First Table Name', 'Second Table Name', 'First Table Foreign Key',
            'Second Table Foreign Key'
        ]
    )
    return spider_schema,spider_primary,spider_foreign

In [55]:
spider_schema,spider_primary,spider_foreign = creating_schema(
    os.path.join(SPIDER_DATASET_DIR, 'tables.json')
)

## Find field

In [56]:
spider_schema.head(20)

,Database name,Table Name,Field Name,Type
0,perpetrator,perpetrator,*,text
1,perpetrator,people,*,text
2,perpetrator,perpetrator,Perpetrator_ID,number
3,perpetrator,perpetrator,People_ID,number
4,perpetrator,perpetrator,Date,text
5,perpetrator,perpetrator,Year,number
6,perpetrator,perpetrator,Location,text
7,perpetrator,perpetrator,Country,text
8,perpetrator,perpetrator,Killed,number
9,perpetrator,perpetrator,Injured,number


In [57]:
spider_schema[' Type'].unique()

array(['text', 'number', 'time', 'others', 'boolean'], dtype=object)

In [58]:
spider_schema[spider_schema[' Type'] == 'others']

,Database name,Table Name,Field Name,Type
849,debate,debate_people,If_Affirmative_Win,others
1490,concert_singer,singer,Is_male,others
2219,school_bus,school_bus,If_full_time,others
2613,apartment_rentals,View_Unit_Status,available_yn,others
3134,employee_hire_evaluation,hiring,Is_full_time,others
3671,sakila_1,staff,picture,others
4477,orchestra,show,If_first_show,others
5344,party_host,party_host,Is_Main_in_Charge,others


In [59]:
def find_fields_MYSQL_like(db_name, spider_schema):
    df = spider_schema[spider_schema['Database name'] == db_name]
    df = df.groupby(' Table Name')
    output = ""
    for name, group in df:
        output += "Table " +name+ ', columns = ['
        for index, row in group.iterrows():
            output += row[" Field Name"]+','
        output = output[:-1]
        output += "]\n"
    return output

field = find_fields_MYSQL_like('perpetrator', spider_schema)
print(field)

Table people, columns = [*,People_ID,Name,Height,Weight,Home Town]
Table perpetrator, columns = [*,Perpetrator_ID,People_ID,Date,Year,Location,Country,Killed,Injured]



## Find foreign key

In [60]:
spider_foreign.head(10)

,Database name,First Table Name,Second Table Name,First Table Foreign Key,Second Table Foreign Key
0,perpetrator,perpetrator,people,People_ID,People_ID
1,college_2,course,department,dept_name,dept_name
2,college_2,instructor,department,dept_name,dept_name
3,college_2,section,classroom,building,building
4,college_2,section,classroom,room_number,room_number
5,college_2,section,course,course_id,course_id
6,college_2,teaches,instructor,ID,ID
7,college_2,teaches,section,course_id,course_id
8,college_2,teaches,section,sec_id,sec_id
9,college_2,teaches,section,semester,semester


In [61]:
def find_foreign_keys_MYSQL_like(db_name, spider_foreign):
    df = spider_foreign[spider_foreign['Database name'] == db_name]
    output = "["
    for index, row in df.iterrows():
        output += row['First Table Name'] + '.' + row['First Table Foreign Key'] + " = " + row['Second Table Name'] + '.' + row['Second Table Foreign Key'] + ',' + '\n'
    output= output[:-1] + "]"
    return output

foreign = find_foreign_keys_MYSQL_like('perpetrator', spider_foreign)
print(foreign)

[perpetrator.People_ID = people.People_ID,]


## Get context

In [69]:
def convert_type_to_sql_type(type):
    if type == 'text':
        return 'VARCHAR'
    elif type == 'integer' or type == 'number' or type == 'int':
        return 'INTEGER'
    elif type == 'time':
        return 'DATETIME'
    elif type == 'boolean':
        return 'BOOLEAN'
    elif type == 'real' or type == 'float' or type == 'double':
        return 'FLOAT'
    elif type == 'others':
        return 'BOOLEAN'
    else:
        return 'VARCHAR'

def get_context_with_db_name(db_name, spider_schema, spider_primary, spider_foreign):
    print("USE " + db_name + ";")
    # find all tables related to db_name
    df = spider_schema[spider_schema['Database name'] == db_name]
    df = df.groupby(' Table Name')
    tables = []
    for name, group in df:
        table = {}
        table['name'] = name
        table['columns'] = []
        for index, row in group.iterrows():
            table['columns'].append((row[" Field Name"], convert_type_to_sql_type(row[' Type'])))
        tables.append(table)
        
    # for each table, create the "CREATE TABLE" statement and append it to context
    statements = []
    for table in tables:
        statement = "CREATE TABLE " + table['name'] + " ("
        for idx, column in enumerate(table['columns']):
            col_name = column[0]
            col_type = column[1]
            if col_name == '*':
                continue
            if ' ' in col_name:
                col_name = '"' + col_name + '"'
            
            
            statement += col_name + " " + col_type
            if idx != len(table['columns']) - 1:
                statement += ", "
        statement = statement + ")"
        statements.append(statement)
    
    # print("; ".join(statements))
    return "; ".join(statements)

get_context_with_db_name('perpetrator', spider_schema, spider_primary, spider_foreign)

USE perpetrator;


'CREATE TABLE people (People_ID INTEGER, Name VARCHAR, Height INTEGER, Weight INTEGER, "Home Town" VARCHAR); CREATE TABLE perpetrator (Perpetrator_ID INTEGER, People_ID INTEGER, Date VARCHAR, Year INTEGER, Location VARCHAR, Country VARCHAR, Killed INTEGER, Injured INTEGER)'

In [71]:
dev_dataset = pd.read_json(os.path.join(SPIDER_DATASET_DIR, 'dev.json'))
dev_dataset.head(10)

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], ..."
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], ..."
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","Show name, country, age for all singers ordere...","[Show, name, ,, country, ,, age, for, all, sin...","{'from': {'table_units': [['table_unit', 1]], ..."
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","What are the names, countries, and ages for ev...","[What, are, the, names, ,, countries, ,, and, ...","{'from': {'table_units': [['table_unit', 1]], ..."
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","What is the average, minimum, and maximum age ...","[What, is, the, average, ,, minimum, ,, and, m...","{'from': {'table_units': [['table_unit', 1]], ..."
5,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","What is the average, minimum, and maximum age ...","[What, is, the, average, ,, minimum, ,, and, m...","{'from': {'table_units': [['table_unit', 1]], ..."
6,concert_singer,"SELECT song_name , song_release_year FROM sin...","[SELECT, song_name, ,, song_release_year, FROM...","[select, song_name, ,, song_release_year, from...",Show the name and the release year of the song...,"[Show, the, name, and, the, release, year, of,...","{'from': {'table_units': [['table_unit', 1]], ..."
7,concert_singer,"SELECT song_name , song_release_year FROM sin...","[SELECT, song_name, ,, song_release_year, FROM...","[select, song_name, ,, song_release_year, from...",What are the names and release years for all t...,"[What, are, the, names, and, release, years, f...","{'from': {'table_units': [['table_unit', 1]], ..."
8,concert_singer,SELECT DISTINCT country FROM singer WHERE age ...,"[SELECT, DISTINCT, country, FROM, singer, WHER...","[select, distinct, country, from, singer, wher...",What are all distinct countries where singers ...,"[What, are, all, distinct, countries, where, s...","{'from': {'table_units': [['table_unit', 1]], ..."
9,concert_singer,SELECT DISTINCT country FROM singer WHERE age ...,"[SELECT, DISTINCT, country, FROM, singer, WHER...","[select, distinct, country, from, singer, wher...",What are the different countries with singers...,"[What, are, the, different, countries, with, s...","{'from': {'table_units': [['table_unit', 1]], ..."
